In [1]:
import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import argparse

Using TensorFlow backend.


In [2]:
strategy = tf.distribute.MirroredStrategy()
G = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [3]:
labels = np.load('labels.npy')
animals = np.load('label_to_animal.npz')
animals = animals['arr_0']
for i in animals:
    print(i)
NUM_CLASSES = animals.shape[0]

bobcat
opossum
empty
coyote
raccoon
bird
dog
cat
squirrel
rabbit
skunk
rodent
badger
deer
car
fox


In [4]:
images = np.load('images.npy')
print(images.shape)
INPUT_SHAPE = images[0].shape

(10000, 384, 512, 3)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1)

In [28]:
# definine the total number of epochs to train for along with the
# initial learning rate
def poly_decay(epoch):
    # initialize the maximum number of epochs, base learning rate,
    # and power of the polynomial
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0
    # compute the new learning rate based on polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
    # return the new learning rate
    return alpha

# construct the image generator for data augmentation and construct
# the set of callbacks
aug = ImageDataGenerator(width_shift_range=0.1,
    height_shift_range=0.1, horizontal_flip=True,
    fill_mode="nearest")
callbacks = [LearningRateScheduler(poly_decay)]

In [7]:
y_lst = list(np.argmax(y_train, axis=1))
unique, counts = np.unique(np.argmax(y_train, axis=1), return_counts=True)
class_weights = {unique[i]:counts[i] for i in range(len(unique))}
for i in range(NUM_CLASSES):
    if i not in class_weights:
        class_weights[i] = 0

In [8]:
base_model = applications.resnet50.ResNet50(weights=None, include_top=False, input_shape=INPUT_SHAPE)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [21]:
DROPOUT = 0.2
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(DROPOUT)(x)
predictions = keras.layers.Dense(NUM_CLASSES, activation= 'softmax')(x)

In [22]:
from keras.optimizers import Adam

INIT_LR = 5e-3
adam = Adam(lr=INIT_LR)

with strategy.scope():
    model = keras.models.Model(inputs = base_model.input, outputs = predictions)
    model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
NUM_EPOCHS = 100
BATCH_SIZE = 10

# train the network
print("[INFO] training network...")
"""H = model.fit_generator(
    aug.flow(X_train, y_train, batch_size=BATCH_SIZE * G),
    validation_data=(X_test, y_test),
    class_weight=class_weights,
    steps_per_epoch=len(X_train) // (BATCH_SIZE * G),
    epochs=NUM_EPOCHS,
    callbacks=callbacks, verbose=2)"""

model.fit(X_train, y_train, epochs = NUM_EPOCHS, batch_size = BATCH_SIZE,
          validation_data=(X_test, y_test), class_weight=class_weights)

[INFO] training network...
Train on 900 samples, validate on 100 samples
Epoch 1/10
900/900 [==============================] - 42s 47ms/step - loss: 187.2141 - accuracy: 0.2878 - val_loss: 50.6827 - val_accuracy: 0.3300
Epoch 2/10
900/900 [==============================] - 42s 47ms/step - loss: 169.5505 - accuracy: 0.2867 - val_loss: 6.0680 - val_accuracy: 0.3300
Epoch 3/10
168/900 [====>.........................] - ETA: 32s - loss: 165.2557 - accuracy: 0.3333

KeyboardInterrupt: 

In [ ]:
print('\n# Evaluate on test data')
results = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print('test loss, test acc:', results)

In [ ]:
model.save("model.h5")